In [411]:
# Import necessary
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mt
import seaborn as sns
import numpy as np
import datetime as dt
import math

In [412]:
# Shows what each ICD9 code stands for 
D_ICD_Diagnoses = pd.read_csv('D_ICD_DIAGNOSES.csv')

# Shows all ICD9 codes for each patient marked by subject ID 
Diagnoses_ICD = pd.read_csv('DIAGNOSES_ICD.csv')

# List of all patients and their information
Patients = pd.read_csv('PATIENTS.csv')

# Additional information for each patient 
Admissions = pd.read_csv('ADMISSIONS.csv')

# Shows lab measurements with a corresponding ITEMID
lab = pd.read_csv('D_LABITEMS.csv')

# Shows results for each ITEMID lab measurements
eve = pd.read_csv('LABEVENTS.csv')

# Shows patient's own body measurements, heart rate, urine output....etc
item = pd.read_csv('D_ITEMS.csv')

# Shows results for patient's output measurements
out = pd.read_csv('OUTPUTEVENTS.csv')

In [413]:
# Calculating parameters for each patient(age of death, discharge..etc)

# Add Patient DOB in usable format and Merge with patient information
Patients_DOB = Patients['DOB']
Patients_DOB2 = []
for n in range(len(Patients_DOB)):
    prelim = Patients_DOB[n]
    Patients_DOB2.append(dt.datetime.strptime(prelim[0:10],'%Y-%m-%d').date())
Birth_Date_Series = pd.Series(Patients_DOB2)
Patients['DOB-2'] = Birth_Date_Series

# Add Data for Patient's Date of Death
Patients_DODeath = Patients['DOD']
Patients_DOD = []
for n in range(len(Patients_DODeath)):
    if pd.isnull(Patients_DODeath[n]) == True:  
        Patients_DOD.append(np.nan)
    else:
        prelim = Patients_DODeath[n]
        Patients_DOD.append(dt.datetime.strptime(prelim[0:10],'%Y-%m-%d').date())

# Add Data for Patient's Date of Admission
Patients_Admit = Admissions['ADMITTIME']
Patients_Admissions = []
for n in range(len(Patients_Admit)):
    if pd.isnull(Patients_Admit[n]) == True:  
        Patients_Admissions.append(np.nan)
    else:
        prelim = Patients_Admit[n]
        Patients_Admissions.append(dt.datetime.strptime(prelim[0:10],'%Y-%m-%d').date())

# Add Data for Patient's Date of Release
Patients_leave = Admissions['DISCHTIME']
Patients_Release = []
for n in range(len(Patients_leave)):
    if pd.isnull(Patients_leave[n]) == True:  
        Patients_Release.append(np.nan)
    else:
        prelim = Patients_leave[n]
        Patients_Release.append(dt.datetime.strptime(prelim[0:10],'%Y-%m-%d').date())
        
# Add Data to Series       
Birth_Date_Series = pd.Series(Patients_DOB2)
Death_Series = pd.Series(Patients_DOD)

Admit_Series = pd.Series(Patients_Admissions)
Release_Series = pd.Series(Patients_Release)

# Find Age at Death
Age_Death = (Death_Series - Birth_Date_Series)
death_age = []

for entry in range(len(Age_Death)):
    if type(Age_Death[entry]) == float:
        death_age.append(float('NAN'))
    else:
        death_age.append(Age_Death[entry].days)

Age_Death = pd.Series(death_age)

# Find Total Admission Times
Admissions_Time = (Release_Series - Admit_Series)

Admit_Time = []

for entry in range(len(Admissions_Time)):
    if type(Admissions_Time[entry]) == float:
        Admit_Time.append(float('NAN'))
    else:
        Admit_Time.append(Admissions_Time[entry].days)

Admissions_Time = pd.Series(Admit_Time)
        
# Add to Pandas Dataframe
Patients['DOB-2'] = Birth_Date_Series
Patients['DOD-2'] = Death_Series
Patients['AOD'] = Age_Death

Admissions['Admit'] = Admit_Series
Admissions['Release'] = Release_Series
Admissions['Total Admission Time'] = Admissions_Time 

Admissions_culled = Admissions.drop_duplicates('SUBJECT_ID', keep = 'last')
Admissions_culled_Admits = Admissions_culled['ADMITTIME']

Admit_Ages = (Admit_Series - Birth_Date_Series)

# Combine admissions information and patient's information
Admissions_culled = Admissions.copy(deep = False)
Admissions_culled = Admissions_culled.drop_duplicates('SUBJECT_ID', keep = 'last')

Admissions_long = Admissions_culled.merge(Patients, on='SUBJECT_ID')
Admissions_long.head()
Admit_Series = pd.Series(Patients_DOD)
Admissions_culled_Admits = Admissions_culled['ADMITTIME']

Admit_times2 = pd.Series(Admissions_long['ADMITTIME'])
Atimes = []
for n in range(len(Admit_times2)):
    if pd.isnull(Admit_times2[n]) == True:  
        Atimes.append(np.nan)
    else:
        prelim = Admit_times2[n]
        Atimes.append(dt.datetime.strptime(prelim[0:10],'%Y-%m-%d').date())

DOB_Admit_times = pd.Series(Admissions_long['DOB'])
DOBAtimes = []
for n in range(len(DOB_Admit_times)):
    if pd.isnull(DOB_Admit_times[n]) == True:  
        DOBAtimes.append(np.nan)
    else:
        prelim = DOB_Admit_times[n]
        DOBAtimes.append(dt.datetime.strptime(prelim[0:10],'%Y-%m-%d').date())

Atimes = pd.Series(Atimes)
DOBAtimes = pd.Series(DOBAtimes)
        
admit_births = (Atimes - DOBAtimes)

Admit_Timet = []
for entry in range(len(admit_births)):
    if type(admit_births[entry]) == float:
        Admit_Timet.append(float('NAN'))
    else:
        Admit_Timet.append(admit_births[entry].days)

Admit_Timet = pd.Series(Admit_Timet)

# Find total number of visits for each subject ID
visit_count = pd.DataFrame(Admissions.SUBJECT_ID.value_counts())
visit_count = pd.DataFrame(Admissions.SUBJECT_ID.value_counts())
visit_count.reset_index(level=0, inplace=True)
visit_count.columns = ['SUBJECT_ID', 'ADMISSIONS']

Admissions_long = Admissions_long.merge(visit_count, on='SUBJECT_ID')

In [414]:
# Find causes of death for patients based on diagnosis
cdeath = []
subjectid = []
ldeath = []
ideath = []
edeath = []
tdeath = []
hamdiddeath = []
admitt = []

for entry in range(len(Admissions)):
    if Admissions['HOSPITAL_EXPIRE_FLAG'][entry] == 0:
        continue
    elif Admissions['HOSPITAL_EXPIRE_FLAG'][entry] == 1:
        subjectid.append(Admissions['SUBJECT_ID'][entry])
        cdeath.append(Admissions['DIAGNOSIS'][entry])
        ldeath.append(Admissions['ADMISSION_LOCATION'][entry])
        tdeath.append(Admissions['ADMISSION_TYPE'][entry])
        ideath.append(Admissions['INSURANCE'][entry])
        edeath.append(Admissions['ETHNICITY'][entry])
        hamdiddeath.append(Admissions['HADM_ID'][entry])
        admitt.append(Admissions['Total Admission Time'][entry])
        
cdeath = pd.Series(cdeath)
subjectid = pd.Series(subjectid)
ldeath = pd.Series(ldeath)
ideath = pd.Series(ideath)
edeath = pd.Series(edeath)
tdeath = pd.Series(tdeath)
hamdiddeath = pd.Series(hamdiddeath)
admitt = pd.Series(admitt)

Death = pd.DataFrame()
Death['SUBJECT_ID'] = subjectid

Death['CAUSE'] = cdeath

Death.head()

,SUBJECT_ID,CAUSE
0,31,STATUS EPILEPTICUS
1,56,HEAD BLEED
2,61,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA
3,67,SUBARACHNOID HEMORRHAGE
4,84,"GLIOBLASTOMA,NAUSEA"


In [415]:
kidney_failure_desc = Death[Death['CAUSE'].str.contains("SEPSIS|CREATININE|MYOCARDIAL|CARDIAC|HEART", na=False)==True]
cancer = Death[Death['CAUSE'].str.contains("CANCER", na=False)==True]
hypo = Death[Death['CAUSE'].str.contains("HYPOTENSION", na=False)==True]
shock = Death[Death['CAUSE'].str.contains("SEPTIC SHOCK", na=False)==True]
anemia = Death[Death['CAUSE'].str.contains("ANEMIA", na=False)==True]
hemo = Death[Death['CAUSE'].str.contains("HEMORRHAGE", na=False)==True]
acid = Death[Death['CAUSE'].str.contains("ACIDOSIS", na=False)==True]
liver = Death[Death['CAUSE'].str.contains("LIVER FAILURE", na=False)==True]



del cancer['CAUSE']
cancer['CANCER_FLAG'] = 1

/Users/brian/.local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [416]:
# Merge admissions information and patients information for a more comprehensive demographics
demographics = pd.DataFrame()
demographics['SUBJECT_ID'] = Admissions_long['SUBJECT_ID']
demographics['GENDER'] = Admissions_long['GENDER']
demographics['DOB'] = Admissions_long['DOB-2']
demographics['DOD'] = Admissions_long['DOD-2']
demographics['DOA'] = Admissions_long['ADMITTIME']
demographics['ADMIT_AGE'] = Admit_Timet
demographics['ETHNICITY'] = Admissions_long['ETHNICITY']
demographics['MARITAL_STATUS'] = Admissions_long['MARITAL_STATUS']
demographics['LANGUAGE'] = Admissions_long['LANGUAGE']
demographics['RELIGION'] = Admissions_long['RELIGION']
demographics['INSURANCE'] = Admissions_long['INSURANCE']
demographics['ADMISSION_LOCATION'] = Admissions_long['ADMISSION_LOCATION']
demographics['#ADMISSIONS'] = Admissions_long['ADMISSIONS']
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,INSURANCE,ADMISSION_LOCATION,#ADMISSIONS
0,22,F,2131-05-07,NaN,2196-04-09 12:26:00,23714,WHITE,MARRIED,NaN,UNOBTAINABLE,Private,EMERGENCY ROOM ADMIT,1
1,23,M,2082-07-17,NaN,2157-10-18 19:34:00,27486,WHITE,MARRIED,ENGL,CATHOLIC,Medicare,TRANSFER FROM HOSP/EXTRAM,2
2,24,M,2100-05-31,NaN,2139-06-06 16:14:00,14250,WHITE,SINGLE,NaN,PROTESTANT QUAKER,Private,TRANSFER FROM HOSP/EXTRAM,1
3,25,M,2101-11-21,NaN,2160-11-02 02:06:00,21531,WHITE,MARRIED,NaN,UNOBTAINABLE,Private,EMERGENCY ROOM ADMIT,1
4,26,M,2054-05-04,2128-02-25,2126-05-06 15:16:00,26299,UNKNOWN/NOT SPECIFIED,SINGLE,NaN,CATHOLIC,Medicare,TRANSFER FROM HOSP/EXTRAM,1


In [417]:
# Merge Death and demographics on subject ID to add relevant data about patient deaths
demographics = pd.merge(Death, demographics, on= 'SUBJECT_ID', how = 'outer')

In [418]:
demographics['DOA'] = pd.to_datetime(demographics['DOA'])
demographics['DOD'] = pd.to_datetime(demographics['DOD'])

In [419]:
# Calculate time it takes for patients to die since admission in hours
demographics['delta'] = demographics['DOD'] - demographics['DOA']    
demographics['delta'] = demographics['delta'] / np.timedelta64(1,'h')

In [420]:
demographics.head()

,SUBJECT_ID,CAUSE,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,INSURANCE,ADMISSION_LOCATION,#ADMISSIONS,delta
0,31,STATUS EPILEPTICUS,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,26394,WHITE,MARRIED,NaN,CATHOLIC,Medicare,TRANSFER FROM HOSP/EXTRAM,1,168.550000
1,56,HEAD BLEED,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,109573,WHITE,NaN,NaN,NOT SPECIFIED,Medicare,EMERGENCY ROOM ADMIT,1,141.983333
2,61,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,20163,WHITE,MARRIED,NaN,CATHOLIC,Private,CLINIC REFERRAL/PREMATURE,2,701.800000
3,67,SUBARACHNOID HEMORRHAGE,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,26842,WHITE,SINGLE,NaN,JEWISH,Medicare,EMERGENCY ROOM ADMIT,2,-0.750000
4,84,"GLIOBLASTOMA,NAUSEA",F,2151-10-21,2196-04-17,2196-04-14 04:02:00,16247,WHITE,MARRIED,NaN,OTHER,Private,EMERGENCY ROOM ADMIT,2,67.966667


In [421]:
# Create a flag for patients outside of hospital where we don't have information on
death_cause = []
outside_death_cause = []
for entry in range(len(demographics)):
    if pd.isnull(demographics['DOD'][entry]) == False and pd.isnull(demographics['CAUSE'][entry]) == True:
        death_cause.append('Death Outside of Hospital')
        outside_death_cause.append(1)
    else: 
        death_cause.append(demographics['CAUSE'][entry])
        outside_death_cause.append(0)
        
death_cause = pd.Series(death_cause)
outside_death_flag = pd.Series(outside_death_cause)

demographics['CAUSE'] = death_cause
demographics['OUTSIDE_DEATH_FLAG'] = outside_death_flag
demographics.head()


# Add death flag within a 1 year frame
death_flag = []
for entry in range(len(demographics)):
    if demographics['delta'][entry] <= 17520:
        death_flag.append(1)
    else: 
        death_flag.append(0)


death_flag = pd.Series(death_flag)
demographics['DEATH_FLAG'] = death_flag

demographics['DEATH_FLAG'] = demographics['DEATH_FLAG'].fillna(0)

# Add too old flag
old_flag = []
for entry in range(len(demographics)):
    if demographics['ADMIT_AGE'][entry] > 32850:
        old_flag.append(1)
    else:
        old_flag.append(0)

old_flag = pd.Series(old_flag)
demographics['OLD_FLAG'] = old_flag

In [422]:
# Leave null for patients too old
age = []
for entry in range(len(demographics)):
     if demographics['OLD_FLAG'][entry] == 0:
            age.append(demographics['ADMIT_AGE'][entry])
     else: 
        age.append(np.nan)
age = pd.Series(age)
demographics['ADMIT_AGE'] = age
demographics.head()

,SUBJECT_ID,CAUSE,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,INSURANCE,ADMISSION_LOCATION,#ADMISSIONS,delta,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG
0,31,STATUS EPILEPTICUS,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,26394.0,WHITE,MARRIED,NaN,CATHOLIC,Medicare,TRANSFER FROM HOSP/EXTRAM,1,168.550000,0,1,0
1,56,HEAD BLEED,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,Medicare,EMERGENCY ROOM ADMIT,1,141.983333,0,1,1
2,61,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,20163.0,WHITE,MARRIED,NaN,CATHOLIC,Private,CLINIC REFERRAL/PREMATURE,2,701.800000,0,1,0
3,67,SUBARACHNOID HEMORRHAGE,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,26842.0,WHITE,SINGLE,NaN,JEWISH,Medicare,EMERGENCY ROOM ADMIT,2,-0.750000,0,1,0
4,84,"GLIOBLASTOMA,NAUSEA",F,2151-10-21,2196-04-17,2196-04-14 04:02:00,16247.0,WHITE,MARRIED,NaN,OTHER,Private,EMERGENCY ROOM ADMIT,2,67.966667,0,1,0


In [423]:
# Add cardiovascular flag to data
demographics = pd.merge(demographics, cancer, on= 'SUBJECT_ID', how = 'outer')
demographics.head()

,SUBJECT_ID,CAUSE,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,INSURANCE,ADMISSION_LOCATION,#ADMISSIONS,delta,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,CANCER_FLAG
0,31,STATUS EPILEPTICUS,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,26394.0,WHITE,MARRIED,NaN,CATHOLIC,Medicare,TRANSFER FROM HOSP/EXTRAM,1,168.550000,0,1,0,NaN
1,56,HEAD BLEED,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,Medicare,EMERGENCY ROOM ADMIT,1,141.983333,0,1,1,NaN
2,61,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,20163.0,WHITE,MARRIED,NaN,CATHOLIC,Private,CLINIC REFERRAL/PREMATURE,2,701.800000,0,1,0,NaN
3,67,SUBARACHNOID HEMORRHAGE,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,26842.0,WHITE,SINGLE,NaN,JEWISH,Medicare,EMERGENCY ROOM ADMIT,2,-0.750000,0,1,0,NaN
4,84,"GLIOBLASTOMA,NAUSEA",F,2151-10-21,2196-04-17,2196-04-14 04:02:00,16247.0,WHITE,MARRIED,NaN,OTHER,Private,EMERGENCY ROOM ADMIT,2,67.966667,0,1,0,NaN


In [424]:
cancer_flag = []

for entry in range(len(demographics)):
    if pd.isnull(demographics['CANCER_FLAG'][entry]) == True:
        cancer_flag.append(1)
    else:
        cancer_flag.append(0)
    

cancer_flag = pd.Series(cancer_flag)
demographics['CANCER_FLAG'] = cancer_flag

In [425]:
# Test whether patients who died actually lived less than or equal to a year since admission
demographics[demographics['DEATH_FLAG'] == 1].head()

,SUBJECT_ID,CAUSE,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,INSURANCE,ADMISSION_LOCATION,#ADMISSIONS,delta,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,CANCER_FLAG
0,31,STATUS EPILEPTICUS,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,26394.0,WHITE,MARRIED,NaN,CATHOLIC,Medicare,TRANSFER FROM HOSP/EXTRAM,1,168.550000,0,1,0,1
1,56,HEAD BLEED,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,Medicare,EMERGENCY ROOM ADMIT,1,141.983333,0,1,1,1
2,61,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,20163.0,WHITE,MARRIED,NaN,CATHOLIC,Private,CLINIC REFERRAL/PREMATURE,2,701.800000,0,1,0,1
3,67,SUBARACHNOID HEMORRHAGE,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,26842.0,WHITE,SINGLE,NaN,JEWISH,Medicare,EMERGENCY ROOM ADMIT,2,-0.750000,0,1,0,1
4,84,"GLIOBLASTOMA,NAUSEA",F,2151-10-21,2196-04-17,2196-04-14 04:02:00,16247.0,WHITE,MARRIED,NaN,OTHER,Private,EMERGENCY ROOM ADMIT,2,67.966667,0,1,0,1


In [426]:
# Extracting all AKI patients
codes = ['5845', '5846', '5847', '5848', '5949', '66930', '66932', '66934']

aki_Diagnoses_ICD = Diagnoses_ICD.loc[Diagnoses_ICD['ICD9_CODE'].str.match('|'.join(codes), na=False)] # All AKI patients
aki_Diagnoses = pd.DataFrame(aki_Diagnoses_ICD.SUBJECT_ID)
aki_Diagnoses = aki_Diagnoses.drop_duplicates()
aki_Diagnoses['AKI_DIAGNOSIS_FLAG'] = 1

# Merge with demographics file:
demographics = pd.merge(demographics, aki_Diagnoses, on= 'SUBJECT_ID', how = 'outer')

In [427]:
# Add AKI diagnosis flags to demographic data 
aki_flag = []
for entry in range(len(demographics)):
    if demographics['AKI_DIAGNOSIS_FLAG'][entry] == 1:
        aki_flag.append(1)
    else:
        aki_flag.append(0)
aki_flag = pd.Series(aki_flag)
demographics['AKI_DIAGNOSIS_FLAG'] = aki_flag

In [428]:
# Add kidney failure cause of death flag to demographic data
demographics = pd.merge(demographics, kidney_failure_desc, on= 'SUBJECT_ID', how = 'outer')
demographics.head()

,SUBJECT_ID,CAUSE_x,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,...,INSURANCE,ADMISSION_LOCATION,#ADMISSIONS,delta,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,CANCER_FLAG,AKI_DIAGNOSIS_FLAG,CAUSE_y
0,31,STATUS EPILEPTICUS,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,26394.0,WHITE,MARRIED,NaN,...,Medicare,TRANSFER FROM HOSP/EXTRAM,1,168.550000,0,1,0,1,0,NaN
1,56,HEAD BLEED,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,...,Medicare,EMERGENCY ROOM ADMIT,1,141.983333,0,1,1,1,0,NaN
2,61,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,20163.0,WHITE,MARRIED,NaN,...,Private,CLINIC REFERRAL/PREMATURE,2,701.800000,0,1,0,1,0,NaN
3,67,SUBARACHNOID HEMORRHAGE,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,26842.0,WHITE,SINGLE,NaN,...,Medicare,EMERGENCY ROOM ADMIT,2,-0.750000,0,1,0,1,0,NaN
4,84,"GLIOBLASTOMA,NAUSEA",F,2151-10-21,2196-04-17,2196-04-14 04:02:00,16247.0,WHITE,MARRIED,NaN,...,Private,EMERGENCY ROOM ADMIT,2,67.966667,0,1,0,1,0,NaN


In [429]:
kidney_failure_flag = []

for entry in range(len(demographics)):
    if pd.isnull(demographics['CAUSE_y'][entry]) == True:
        kidney_failure_flag.append(0)
    else:
        kidney_failure_flag.append(1)

kidney_failure_flag = pd.Series(kidney_failure_flag)
demographics['KIDNEY_FAILURE_FLAG'] = aki_flag

del demographics['CAUSE_y']

demographics['CAUSE'] = demographics['CAUSE_x']
del demographics['CAUSE_x']
demographics['KIDNEY_FAILURE_FLAG'] = kidney_failure_flag

In [430]:
# Add hypotension cause of death flag to demographic data
demographics = pd.merge(demographics, hypo, on= 'SUBJECT_ID', how = 'outer')
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,#ADMISSIONS,delta,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,CANCER_FLAG,AKI_DIAGNOSIS_FLAG,KIDNEY_FAILURE_FLAG,CAUSE_x,CAUSE_y
0,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,26394.0,WHITE,MARRIED,NaN,CATHOLIC,...,1,168.550000,0,1,0,1,0,0,STATUS EPILEPTICUS,NaN
1,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,1,141.983333,0,1,1,1,0,0,HEAD BLEED,NaN
2,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,20163.0,WHITE,MARRIED,NaN,CATHOLIC,...,2,701.800000,0,1,0,1,0,0,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,NaN
3,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,26842.0,WHITE,SINGLE,NaN,JEWISH,...,2,-0.750000,0,1,0,1,0,0,SUBARACHNOID HEMORRHAGE,NaN
4,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,16247.0,WHITE,MARRIED,NaN,OTHER,...,2,67.966667,0,1,0,1,0,0,"GLIOBLASTOMA,NAUSEA",NaN


In [431]:
hypo_flag = []

for entry in range(len(demographics)):
    if pd.isnull(demographics['CAUSE_y'][entry]) == True:
        hypo_flag.append(0)
    else:
        hypo_flag.append(1)

hypo_flag = pd.Series(hypo_flag)
demographics['HYPO_FLAG'] = aki_flag

del demographics['CAUSE_y']

demographics['CAUSE'] = demographics['CAUSE_x']
del demographics['CAUSE_x']
demographics['HYPO_FLAG'] = hypo_flag

In [432]:
# Add septic shock cause of death flag to demographic data
demographics = pd.merge(demographics, shock, on= 'SUBJECT_ID', how = 'outer')
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,delta,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,CANCER_FLAG,AKI_DIAGNOSIS_FLAG,KIDNEY_FAILURE_FLAG,HYPO_FLAG,CAUSE_x,CAUSE_y
0,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,26394.0,WHITE,MARRIED,NaN,CATHOLIC,...,168.550000,0,1,0,1,0,0,0,STATUS EPILEPTICUS,NaN
1,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,141.983333,0,1,1,1,0,0,0,HEAD BLEED,NaN
2,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,20163.0,WHITE,MARRIED,NaN,CATHOLIC,...,701.800000,0,1,0,1,0,0,0,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,NaN
3,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,26842.0,WHITE,SINGLE,NaN,JEWISH,...,-0.750000,0,1,0,1,0,0,0,SUBARACHNOID HEMORRHAGE,NaN
4,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,16247.0,WHITE,MARRIED,NaN,OTHER,...,67.966667,0,1,0,1,0,0,0,"GLIOBLASTOMA,NAUSEA",NaN


In [433]:
shock_flag = []

for entry in range(len(demographics)):
    if pd.isnull(demographics['CAUSE_y'][entry]) == True:
        shock_flag.append(0)
    else:
        shock_flag.append(1)

shock_flag = pd.Series(shock_flag)
demographics['SHOCK_FLAG'] = aki_flag

del demographics['CAUSE_y']

demographics['CAUSE'] = demographics['CAUSE_x']
del demographics['CAUSE_x']
demographics['SHOCK_FLAG'] = shock_flag

In [434]:
# Add anemia cause of death flag to demographic data
demographics = pd.merge(demographics, anemia, on= 'SUBJECT_ID', how = 'outer')
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,CANCER_FLAG,AKI_DIAGNOSIS_FLAG,KIDNEY_FAILURE_FLAG,HYPO_FLAG,SHOCK_FLAG,CAUSE_x,CAUSE_y
0,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,26394.0,WHITE,MARRIED,NaN,CATHOLIC,...,0,1,0,1,0,0,0,0,STATUS EPILEPTICUS,NaN
1,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,0,1,1,1,0,0,0,0,HEAD BLEED,NaN
2,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,20163.0,WHITE,MARRIED,NaN,CATHOLIC,...,0,1,0,1,0,0,0,0,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,NaN
3,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,26842.0,WHITE,SINGLE,NaN,JEWISH,...,0,1,0,1,0,0,0,0,SUBARACHNOID HEMORRHAGE,NaN
4,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,16247.0,WHITE,MARRIED,NaN,OTHER,...,0,1,0,1,0,0,0,0,"GLIOBLASTOMA,NAUSEA",NaN


In [435]:
anemia_flag = []

for entry in range(len(demographics)):
    if pd.isnull(demographics['CAUSE_y'][entry]) == True:
        anemia_flag.append(0)
    else:
        anemia_flag.append(1)

anemia_flag = pd.Series(anemia_flag)
demographics['ANEMIA_FLAG'] = aki_flag

del demographics['CAUSE_y']

demographics['CAUSE'] = demographics['CAUSE_x']
del demographics['CAUSE_x']
demographics['ANEMIA_FLAG'] = anemia_flag

In [436]:
# Add hemorrhage cause of death flag to demographic data
demographics = pd.merge(demographics, hemo, on= 'SUBJECT_ID', how = 'outer')
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,DEATH_FLAG,OLD_FLAG,CANCER_FLAG,AKI_DIAGNOSIS_FLAG,KIDNEY_FAILURE_FLAG,HYPO_FLAG,SHOCK_FLAG,ANEMIA_FLAG,CAUSE_x,CAUSE_y
0,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,26394.0,WHITE,MARRIED,NaN,CATHOLIC,...,1,0,1,0,0,0,0,0,STATUS EPILEPTICUS,NaN
1,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,1,1,1,0,0,0,0,0,HEAD BLEED,NaN
2,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,20163.0,WHITE,MARRIED,NaN,CATHOLIC,...,1,0,1,0,0,0,0,0,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,NaN
3,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,26842.0,WHITE,SINGLE,NaN,JEWISH,...,1,0,1,0,0,0,0,0,SUBARACHNOID HEMORRHAGE,SUBARACHNOID HEMORRHAGE
4,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,16247.0,WHITE,MARRIED,NaN,OTHER,...,1,0,1,0,0,0,0,0,"GLIOBLASTOMA,NAUSEA",NaN


In [437]:
hemo_flag = []

for entry in range(len(demographics)):
    if pd.isnull(demographics['CAUSE_y'][entry]) == True:
        hemo_flag.append(0)
    else:
        hemo_flag.append(1)

hemo_flag = pd.Series(hemo_flag)
demographics['HEMO_FLAG'] = aki_flag

del demographics['CAUSE_y']

demographics['CAUSE'] = demographics['CAUSE_x']
del demographics['CAUSE_x']
demographics['HEMO_FLAG'] = hemo_flag

In [438]:
# Add acidosis cause of death flag to demographic data
demographics = pd.merge(demographics, acid, on= 'SUBJECT_ID', how = 'outer')
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,OLD_FLAG,CANCER_FLAG,AKI_DIAGNOSIS_FLAG,KIDNEY_FAILURE_FLAG,HYPO_FLAG,SHOCK_FLAG,ANEMIA_FLAG,HEMO_FLAG,CAUSE_x,CAUSE_y
0,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,26394.0,WHITE,MARRIED,NaN,CATHOLIC,...,0,1,0,0,0,0,0,0,STATUS EPILEPTICUS,NaN
1,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,1,1,0,0,0,0,0,0,HEAD BLEED,NaN
2,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,20163.0,WHITE,MARRIED,NaN,CATHOLIC,...,0,1,0,0,0,0,0,0,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,NaN
3,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,26842.0,WHITE,SINGLE,NaN,JEWISH,...,0,1,0,0,0,0,0,1,SUBARACHNOID HEMORRHAGE,NaN
4,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,16247.0,WHITE,MARRIED,NaN,OTHER,...,0,1,0,0,0,0,0,0,"GLIOBLASTOMA,NAUSEA",NaN


In [439]:
acid_flag = []

for entry in range(len(demographics)):
    if pd.isnull(demographics['CAUSE_y'][entry]) == True:
        acid_flag.append(0)
    else:
        acid_flag.append(1)

acid_flag = pd.Series(acid_flag)
demographics['ACID_FLAG'] = aki_flag

del demographics['CAUSE_y']

demographics['CAUSE'] = demographics['CAUSE_x']
del demographics['CAUSE_x']
demographics['ACID_FLAG'] = acid_flag

In [440]:
# Add liver failure cause of death flag to demographic data
demographics = pd.merge(demographics, liver, on= 'SUBJECT_ID', how = 'outer')
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,CANCER_FLAG,AKI_DIAGNOSIS_FLAG,KIDNEY_FAILURE_FLAG,HYPO_FLAG,SHOCK_FLAG,ANEMIA_FLAG,HEMO_FLAG,ACID_FLAG,CAUSE_x,CAUSE_y
0,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,26394.0,WHITE,MARRIED,NaN,CATHOLIC,...,1,0,0,0,0,0,0,0,STATUS EPILEPTICUS,NaN
1,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,1,0,0,0,0,0,0,0,HEAD BLEED,NaN
2,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,20163.0,WHITE,MARRIED,NaN,CATHOLIC,...,1,0,0,0,0,0,0,0,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,NaN
3,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,26842.0,WHITE,SINGLE,NaN,JEWISH,...,1,0,0,0,0,0,1,0,SUBARACHNOID HEMORRHAGE,NaN
4,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,16247.0,WHITE,MARRIED,NaN,OTHER,...,1,0,0,0,0,0,0,0,"GLIOBLASTOMA,NAUSEA",NaN


In [441]:
liver_flag = []

for entry in range(len(demographics)):
    if pd.isnull(demographics['CAUSE_y'][entry]) == True:
        liver_flag.append(0)
    else:
        liver_flag.append(1)
        
liver_flag = pd.Series(liver_flag)
demographics['LIVER_FLAG'] = aki_flag

del demographics['CAUSE_y']

demographics['CAUSE'] = demographics['CAUSE_x']
del demographics['CAUSE_x']
demographics['LIVER_FLAG'] = liver_flag

# acid_flag = []

# for entry in range(len(demographics)):
#     if pd.isnull(demographics['CAUSE_y'][entry]) == True:
#         acid_flag.append(0)
#     else:
#         acid_flag.append(1)

# acid_flag = pd.Series(acid_flag)
# demographics['ACID_FLAG'] = aki_flag

# del demographics['CAUSE_y']

# demographics['CAUSE'] = demographics['CAUSE_x']
# del demographics['CAUSE_x']
# demographics['ACID_FLAG'] = acid_flag

In [442]:
# Add age in years to simplify interpretation
demographics['ADMIT_AGE'] = demographics['ADMIT_AGE']/365

In [443]:
# Test
demographics[(demographics['LIVER_FLAG'] == 1) & (demographics['AKI_DIAGNOSIS_FLAG'] == 1) ]

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,CANCER_FLAG,AKI_DIAGNOSIS_FLAG,KIDNEY_FAILURE_FLAG,HYPO_FLAG,SHOCK_FLAG,ANEMIA_FLAG,HEMO_FLAG,ACID_FLAG,LIVER_FLAG,CAUSE
315,2553,M,2076-12-16,2122-12-10,2122-10-31 20:30:00,45.901370,UNKNOWN/NOT SPECIFIED,SEPARATED,NaN,UNOBTAINABLE,...,1,1,0,0,0,0,0,0,1,LIVER FAILURE
461,3764,M,2113-12-13,2140-03-16,2140-02-10 20:33:00,26.178082,UNKNOWN/NOT SPECIFIED,SINGLE,NaN,UNOBTAINABLE,...,1,1,0,0,0,0,0,0,1,LIVER FAILURE
986,9808,F,2072-07-10,2130-03-17,2130-02-03 17:28:00,57.605479,WHITE,MARRIED,NaN,CATHOLIC,...,1,1,0,0,0,0,0,0,1,LIVER FAILURE;CIRRHOSIS
1137,12533,M,2108-08-28,2170-07-04,2170-05-31 17:01:00,61.797260,WHITE,MARRIED,NaN,NOT SPECIFIED,...,1,1,0,0,0,0,0,0,1,LIVER FAILURE
1206,11819,M,2059-10-18,2111-10-23,2111-10-12 19:56:00,52.016438,WHITE,DIVORCED,NaN,CATHOLIC,...,1,1,0,0,0,0,0,0,1,LIVER FAILURE
1314,10126,F,2127-06-04,2171-08-16,2171-07-12 06:02:00,44.134247,WHITE,SINGLE,NaN,UNOBTAINABLE,...,1,1,0,0,0,0,0,0,1,LIVER FAILURE
1401,11106,M,2068-09-07,2110-05-15,2110-05-05 14:51:00,41.682192,BLACK/AFRICAN AMERICAN,SINGLE,NaN,PROTESTANT QUAKER,...,1,1,0,0,0,0,0,0,1,LIVER FAILURE
1477,14533,F,2121-05-09,2160-07-06,2160-06-24 01:43:00,39.153425,WHITE,SINGLE,NaN,CATHOLIC,...,1,1,0,0,0,0,0,0,1,ACUTE/CHRONIC LIVER FAILURE
1492,12909,F,2078-07-10,2125-02-20,2125-02-10 15:19:00,46.619178,UNKNOWN/NOT SPECIFIED,MARRIED,NaN,CATHOLIC,...,1,1,0,0,0,0,0,0,1,LIVER FAILURE
1564,12246,F,2049-07-01,2116-05-24,2116-05-06 20:40:00,66.890411,UNKNOWN/NOT SPECIFIED,WIDOWED,NaN,CATHOLIC,...,1,1,0,0,0,0,0,0,1,LIVER FAILURE


In [444]:
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,CANCER_FLAG,AKI_DIAGNOSIS_FLAG,KIDNEY_FAILURE_FLAG,HYPO_FLAG,SHOCK_FLAG,ANEMIA_FLAG,HEMO_FLAG,ACID_FLAG,LIVER_FLAG,CAUSE
0,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,1,0,0,0,0,0,0,0,0,STATUS EPILEPTICUS
1,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,1,0,0,0,0,0,0,0,0,HEAD BLEED
2,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,55.241096,WHITE,MARRIED,NaN,CATHOLIC,...,1,0,0,0,0,0,0,0,0,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA
3,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,73.539726,WHITE,SINGLE,NaN,JEWISH,...,1,0,0,0,0,0,1,0,0,SUBARACHNOID HEMORRHAGE
4,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,44.512329,WHITE,MARRIED,NaN,OTHER,...,1,0,0,0,0,0,0,0,0,"GLIOBLASTOMA,NAUSEA"


In [445]:
# Add creatinine level feature: first, find serum creatinine's ITEMID, which is 50912
lab[(lab['LABEL']=='Creatinine') & (lab['FLUID']=='Blood')]

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
239,113,50912,Creatinine,Blood,Chemistry,2160-0


In [446]:
# find creatinine level results for each patient
eve2 = eve[eve['ITEMID']==50912]

In [447]:
eve2.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
15,296,3,NaN,50912,2101-10-13 03:00:00,1.7,1.7,mg/dL,abnormal
28,309,3,NaN,50912,2101-10-13 15:47:00,1.5,1.5,mg/dL,abnormal
42,323,3,NaN,50912,2101-10-14 03:00:00,1.4,1.4,mg/dL,abnormal
86,367,3,NaN,50912,2101-10-15 03:30:00,1.6,1.6,mg/dL,abnormal
106,387,3,NaN,50912,2101-10-15 12:15:00,1.5,1.5,mg/dL,abnormal


In [448]:
# merge creatinine level results with demographics
demographics = pd.merge(demographics,eve2[['SUBJECT_ID', 'ITEMID']], on='SUBJECT_ID', how='left' )

In [449]:
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,AKI_DIAGNOSIS_FLAG,KIDNEY_FAILURE_FLAG,HYPO_FLAG,SHOCK_FLAG,ANEMIA_FLAG,HEMO_FLAG,ACID_FLAG,LIVER_FLAG,CAUSE,ITEMID
0,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,0,0,0,STATUS EPILEPTICUS,50912.0
1,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,0,0,0,STATUS EPILEPTICUS,50912.0
2,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,0,0,0,STATUS EPILEPTICUS,50912.0
3,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,0,0,0,STATUS EPILEPTICUS,50912.0
4,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,0,0,0,STATUS EPILEPTICUS,50912.0


In [450]:
demographics = pd.merge(demographics,eve2[['SUBJECT_ID', 'VALUENUM']], on='SUBJECT_ID', how='left' )

In [451]:
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,KIDNEY_FAILURE_FLAG,HYPO_FLAG,SHOCK_FLAG,ANEMIA_FLAG,HEMO_FLAG,ACID_FLAG,LIVER_FLAG,CAUSE,ITEMID,VALUENUM
0,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,0,0,STATUS EPILEPTICUS,50912.0,0.9
1,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,0,0,STATUS EPILEPTICUS,50912.0,0.7
2,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,0,0,STATUS EPILEPTICUS,50912.0,0.9
3,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,0,0,STATUS EPILEPTICUS,50912.0,0.8
4,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,0,0,STATUS EPILEPTICUS,50912.0,0.7


In [452]:
# demographics = demographics[demographics['VALUENUM'].notna()]

In [453]:
demographics = demographics.drop_duplicates('SUBJECT_ID', keep='last') 

In [454]:
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,KIDNEY_FAILURE_FLAG,HYPO_FLAG,SHOCK_FLAG,ANEMIA_FLAG,HEMO_FLAG,ACID_FLAG,LIVER_FLAG,CAUSE,ITEMID,VALUENUM
143,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,0,0,STATUS EPILEPTICUS,50912.0,1.5
159,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,0,0,0,0,0,0,0,HEAD BLEED,50912.0,0.8
14559,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,55.241096,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,0,0,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,50912.0,0.5
14560,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,73.539726,WHITE,SINGLE,NaN,JEWISH,...,0,0,0,0,1,0,0,SUBARACHNOID HEMORRHAGE,50912.0,1.6
14609,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,44.512329,WHITE,MARRIED,NaN,OTHER,...,0,0,0,0,0,0,0,"GLIOBLASTOMA,NAUSEA",50912.0,0.5


In [455]:
demographics.columns = demographics.columns.str.replace('VALUENUM','VALUENUM-1')
demographics.columns = demographics.columns.str.replace('ITEMID','ITEMID-1')

In [456]:
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,KIDNEY_FAILURE_FLAG,HYPO_FLAG,SHOCK_FLAG,ANEMIA_FLAG,HEMO_FLAG,ACID_FLAG,LIVER_FLAG,CAUSE,ITEMID-1,VALUENUM-1
143,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,0,0,STATUS EPILEPTICUS,50912.0,1.5
159,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,0,0,0,0,0,0,0,HEAD BLEED,50912.0,0.8
14559,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,55.241096,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,0,0,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,50912.0,0.5
14560,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,73.539726,WHITE,SINGLE,NaN,JEWISH,...,0,0,0,0,1,0,0,SUBARACHNOID HEMORRHAGE,50912.0,1.6
14609,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,44.512329,WHITE,MARRIED,NaN,OTHER,...,0,0,0,0,0,0,0,"GLIOBLASTOMA,NAUSEA",50912.0,0.5


In [457]:
# Add feature: 24 hr Creatinine level in Urine
lab[(lab['LABEL']=='24 hr Creatinine') & (lab['FLUID']=='Urine')]

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
393,267,51067,24 hr Creatinine,Urine,Chemistry,2162-6


In [458]:
eve3 = eve[eve['ITEMID']==51067]

In [459]:
eve3.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
34571,33629,68,108329.0,51067,2174-01-06 08:10:00,488,488.0,mg/24hr,NaN
54444,62259,115,NaN,51067,2194-10-07 21:51:00,612,612.0,mg/24hr,NaN
57296,54600,109,110668.0,51067,2140-08-29 10:38:00,630,630.0,mg/24hr,NaN
57555,53312,109,128755.0,51067,2138-04-18 10:44:00,284,284.0,mg/24hours,NaN
64535,64117,115,NaN,51067,2194-11-24 04:02:00,340,340.0,mg/24hr,NaN


In [460]:
# merge creatinine level results with demographics
demographics = pd.merge(demographics,eve3[['SUBJECT_ID', 'ITEMID']], on='SUBJECT_ID', how='left' )

In [461]:
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,HYPO_FLAG,SHOCK_FLAG,ANEMIA_FLAG,HEMO_FLAG,ACID_FLAG,LIVER_FLAG,CAUSE,ITEMID-1,VALUENUM-1,ITEMID
0,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,0,STATUS EPILEPTICUS,50912.0,1.5,NaN
1,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,0,0,0,0,0,0,HEAD BLEED,50912.0,0.8,NaN
2,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,55.241096,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,0,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,50912.0,0.5,NaN
3,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,73.539726,WHITE,SINGLE,NaN,JEWISH,...,0,0,0,1,0,0,SUBARACHNOID HEMORRHAGE,50912.0,1.6,NaN
4,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,44.512329,WHITE,MARRIED,NaN,OTHER,...,0,0,0,0,0,0,"GLIOBLASTOMA,NAUSEA",50912.0,0.5,NaN


In [462]:
demographics = pd.merge(demographics,eve3[['SUBJECT_ID', 'VALUENUM']], on='SUBJECT_ID', how='left' )

In [463]:
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,SHOCK_FLAG,ANEMIA_FLAG,HEMO_FLAG,ACID_FLAG,LIVER_FLAG,CAUSE,ITEMID-1,VALUENUM-1,ITEMID,VALUENUM
0,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,STATUS EPILEPTICUS,50912.0,1.5,NaN,NaN
1,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,0,0,0,0,0,HEAD BLEED,50912.0,0.8,NaN,NaN
2,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,55.241096,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,50912.0,0.5,NaN,NaN
3,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,73.539726,WHITE,SINGLE,NaN,JEWISH,...,0,0,1,0,0,SUBARACHNOID HEMORRHAGE,50912.0,1.6,NaN,NaN
4,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,44.512329,WHITE,MARRIED,NaN,OTHER,...,0,0,0,0,0,"GLIOBLASTOMA,NAUSEA",50912.0,0.5,NaN,NaN


In [464]:
# demographics = demographics[demographics['VALUENUM'].notna()]

In [465]:
demographics = demographics.drop_duplicates('SUBJECT_ID', keep='last')

In [466]:
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,SHOCK_FLAG,ANEMIA_FLAG,HEMO_FLAG,ACID_FLAG,LIVER_FLAG,CAUSE,ITEMID-1,VALUENUM-1,ITEMID,VALUENUM
0,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,STATUS EPILEPTICUS,50912.0,1.5,NaN,NaN
1,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,0,0,0,0,0,HEAD BLEED,50912.0,0.8,NaN,NaN
2,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,55.241096,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,50912.0,0.5,NaN,NaN
3,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,73.539726,WHITE,SINGLE,NaN,JEWISH,...,0,0,1,0,0,SUBARACHNOID HEMORRHAGE,50912.0,1.6,NaN,NaN
4,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,44.512329,WHITE,MARRIED,NaN,OTHER,...,0,0,0,0,0,"GLIOBLASTOMA,NAUSEA",50912.0,0.5,NaN,NaN


In [467]:
demographics.columns = demographics.columns.str.replace('ITEMID','ITEMID-2')
demographics.columns = demographics.columns.str.replace('VALUENUM','VALUENUM-2')

In [468]:
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,SHOCK_FLAG,ANEMIA_FLAG,HEMO_FLAG,ACID_FLAG,LIVER_FLAG,CAUSE,ITEMID-2-1,VALUENUM-2-1,ITEMID-2,VALUENUM-2
0,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,STATUS EPILEPTICUS,50912.0,1.5,NaN,NaN
1,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,0,0,0,0,0,HEAD BLEED,50912.0,0.8,NaN,NaN
2,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,55.241096,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,50912.0,0.5,NaN,NaN
3,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,73.539726,WHITE,SINGLE,NaN,JEWISH,...,0,0,1,0,0,SUBARACHNOID HEMORRHAGE,50912.0,1.6,NaN,NaN
4,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,44.512329,WHITE,MARRIED,NaN,OTHER,...,0,0,0,0,0,"GLIOBLASTOMA,NAUSEA",50912.0,0.5,NaN,NaN


In [469]:
demographics['VALUENUM-2-1'] = demographics['VALUENUM-2-1'].fillna(0)
demographics['VALUENUM-2'] = demographics['VALUENUM-2'].fillna(0)
demographics['ITEMID-2-1'] = demographics['ITEMID-2-1'].fillna(0)
demographics['ITEMID-2'] = demographics['ITEMID-2'].fillna(0)

In [470]:
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,SHOCK_FLAG,ANEMIA_FLAG,HEMO_FLAG,ACID_FLAG,LIVER_FLAG,CAUSE,ITEMID-2-1,VALUENUM-2-1,ITEMID-2,VALUENUM-2
0,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,STATUS EPILEPTICUS,50912.0,1.5,0.0,0.0
1,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,0,0,0,0,0,HEAD BLEED,50912.0,0.8,0.0,0.0
2,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,55.241096,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,0,0,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,50912.0,0.5,0.0,0.0
3,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,73.539726,WHITE,SINGLE,NaN,JEWISH,...,0,0,1,0,0,SUBARACHNOID HEMORRHAGE,50912.0,1.6,0.0,0.0
4,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,44.512329,WHITE,MARRIED,NaN,OTHER,...,0,0,0,0,0,"GLIOBLASTOMA,NAUSEA",50912.0,0.5,0.0,0.0


In [471]:
# Body fluids: creatinine
eve4 = eve[eve['ITEMID']==51032]

In [472]:
demographics = pd.merge(demographics,eve4[['SUBJECT_ID', 'ITEMID']], on='SUBJECT_ID', how='left' )

In [473]:
demographics = demographics.drop_duplicates('SUBJECT_ID', keep='last')

In [474]:
demographics = pd.merge(demographics,eve4[['SUBJECT_ID', 'VALUENUM']], on='SUBJECT_ID', how='left' )

In [475]:
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,HEMO_FLAG,ACID_FLAG,LIVER_FLAG,CAUSE,ITEMID-2-1,VALUENUM-2-1,ITEMID-2,VALUENUM-2,ITEMID,VALUENUM
0,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,STATUS EPILEPTICUS,50912.0,1.5,0.0,0.0,NaN,NaN
1,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,0,0,0,HEAD BLEED,50912.0,0.8,0.0,0.0,NaN,NaN
2,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,55.241096,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,50912.0,0.5,0.0,0.0,NaN,NaN
3,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,73.539726,WHITE,SINGLE,NaN,JEWISH,...,1,0,0,SUBARACHNOID HEMORRHAGE,50912.0,1.6,0.0,0.0,NaN,NaN
4,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,44.512329,WHITE,MARRIED,NaN,OTHER,...,0,0,0,"GLIOBLASTOMA,NAUSEA",50912.0,0.5,0.0,0.0,NaN,NaN


In [476]:
demographics.columns = demographics.columns.str.replace('ITEMID','ITEMID-3')
demographics.columns = demographics.columns.str.replace('VALUENUM','VALUENUM-3')

In [477]:
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,HEMO_FLAG,ACID_FLAG,LIVER_FLAG,CAUSE,ITEMID-3-2-1,VALUENUM-3-2-1,ITEMID-3-2,VALUENUM-3-2,ITEMID-3,VALUENUM-3
0,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,STATUS EPILEPTICUS,50912.0,1.5,0.0,0.0,NaN,NaN
1,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,0,0,0,HEAD BLEED,50912.0,0.8,0.0,0.0,NaN,NaN
2,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,55.241096,WHITE,MARRIED,NaN,CATHOLIC,...,0,0,0,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,50912.0,0.5,0.0,0.0,NaN,NaN
3,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,73.539726,WHITE,SINGLE,NaN,JEWISH,...,1,0,0,SUBARACHNOID HEMORRHAGE,50912.0,1.6,0.0,0.0,NaN,NaN
4,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,44.512329,WHITE,MARRIED,NaN,OTHER,...,0,0,0,"GLIOBLASTOMA,NAUSEA",50912.0,0.5,0.0,0.0,NaN,NaN


In [478]:
# Pleural: creatinine
eve5 = eve[eve['ITEMID']==51052]

In [479]:
demographics = pd.merge(demographics,eve5[['SUBJECT_ID', 'ITEMID']], on='SUBJECT_ID', how='left' )
demographics = demographics.drop_duplicates('SUBJECT_ID', keep='last')

In [480]:
demographics = pd.merge(demographics,eve5[['SUBJECT_ID', 'VALUENUM']], on='SUBJECT_ID', how='left' )

In [481]:
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,LIVER_FLAG,CAUSE,ITEMID-3-2-1,VALUENUM-3-2-1,ITEMID-3-2,VALUENUM-3-2,ITEMID-3,VALUENUM-3,ITEMID,VALUENUM
0,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0,STATUS EPILEPTICUS,50912.0,1.5,0.0,0.0,NaN,NaN,NaN,NaN
1,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,0,HEAD BLEED,50912.0,0.8,0.0,0.0,NaN,NaN,NaN,NaN
2,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,55.241096,WHITE,MARRIED,NaN,CATHOLIC,...,0,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,50912.0,0.5,0.0,0.0,NaN,NaN,NaN,NaN
3,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,73.539726,WHITE,SINGLE,NaN,JEWISH,...,0,SUBARACHNOID HEMORRHAGE,50912.0,1.6,0.0,0.0,NaN,NaN,NaN,NaN
4,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,44.512329,WHITE,MARRIED,NaN,OTHER,...,0,"GLIOBLASTOMA,NAUSEA",50912.0,0.5,0.0,0.0,NaN,NaN,NaN,NaN


In [482]:
demographics['ITEMID'] = demographics['ITEMID'].fillna(0)
demographics['VALUENUM'] = demographics['VALUENUM'].fillna(0)
demographics['ITEMID-3'] = demographics['ITEMID-3'].fillna(0)
demographics['VALUENUM-3'] = demographics['VALUENUM-3'].fillna(0)

In [483]:
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,LIVER_FLAG,CAUSE,ITEMID-3-2-1,VALUENUM-3-2-1,ITEMID-3-2,VALUENUM-3-2,ITEMID-3,VALUENUM-3,ITEMID,VALUENUM
0,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0,STATUS EPILEPTICUS,50912.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0
1,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,0,HEAD BLEED,50912.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0
2,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,55.241096,WHITE,MARRIED,NaN,CATHOLIC,...,0,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,50912.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0
3,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,73.539726,WHITE,SINGLE,NaN,JEWISH,...,0,SUBARACHNOID HEMORRHAGE,50912.0,1.6,0.0,0.0,0.0,0.0,0.0,0.0
4,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,44.512329,WHITE,MARRIED,NaN,OTHER,...,0,"GLIOBLASTOMA,NAUSEA",50912.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0


In [484]:
# Add Prothrombin Time (PT) feature, ITEMID = 51274
eve6 = eve[eve['ITEMID'] == 51274]

In [485]:
demographics.columns = demographics.columns.str.replace('VALUENUM','VALUENUM-4')

In [486]:
demographics = pd.merge(demographics,eve6[['SUBJECT_ID', 'VALUENUM']], on='SUBJECT_ID', how='left' )

In [487]:
demographics = demographics.drop_duplicates('SUBJECT_ID', keep='last')

In [488]:
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,CAUSE,ITEMID-3-2-1,VALUENUM-4-3-2-1,ITEMID-3-2,VALUENUM-4-3-2,ITEMID-3,VALUENUM-4-3,ITEMID,VALUENUM-4,VALUENUM
7,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,STATUS EPILEPTICUS,50912.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,13.3
9,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,HEAD BLEED,50912.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0,13.2
79,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,55.241096,WHITE,MARRIED,NaN,CATHOLIC,...,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,50912.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,12.2
80,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,73.539726,WHITE,SINGLE,NaN,JEWISH,...,SUBARACHNOID HEMORRHAGE,50912.0,1.6,0.0,0.0,0.0,0.0,0.0,0.0,13.2
83,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,44.512329,WHITE,MARRIED,NaN,OTHER,...,"GLIOBLASTOMA,NAUSEA",50912.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,12.9


In [489]:
# Add hemotocrit feature, ITEMID = 51221
eve7 = eve[eve['ITEMID'] == 51221]

In [490]:
demographics.columns = demographics.columns.str.replace('VALUENUM','VALUENUM-5')

In [491]:
demographics = pd.merge(demographics,eve7[['SUBJECT_ID', 'VALUENUM']], on='SUBJECT_ID', how='left' )

In [492]:
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,ITEMID-3-2-1,VALUENUM-5-4-3-2-1,ITEMID-3-2,VALUENUM-5-4-3-2,ITEMID-3,VALUENUM-5-4-3,ITEMID,VALUENUM-5-4,VALUENUM-5,VALUENUM
0,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,50912.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,13.3,30.0
1,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,50912.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,13.3,28.2
2,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,50912.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,13.3,29.7
3,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,50912.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,13.3,31.3
4,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,50912.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,13.3,29.8


In [493]:
demographics = demographics.drop_duplicates('SUBJECT_ID', keep='last')

In [494]:
# Add hemoglobin feature, ITEMID = 51222
eve8 = eve[eve['ITEMID'] == 51222]

In [495]:
demographics.columns = demographics.columns.str.replace('VALUENUM','VALUENUM-6')

In [496]:
demographics = pd.merge(demographics,eve8[['SUBJECT_ID', 'VALUENUM']], on='SUBJECT_ID', how='left' )

In [497]:
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,VALUENUM-6-5-4-3-2-1,ITEMID-3-2,VALUENUM-6-5-4-3-2,ITEMID-3,VALUENUM-6-5-4-3,ITEMID,VALUENUM-6-5-4,VALUENUM-6-5,VALUENUM-6,VALUENUM
0,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,1.5,0.0,0.0,0.0,0.0,0.0,0.0,13.3,33.9,10.6
1,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,1.5,0.0,0.0,0.0,0.0,0.0,0.0,13.3,33.9,9.5
2,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,1.5,0.0,0.0,0.0,0.0,0.0,0.0,13.3,33.9,10.1
3,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,1.5,0.0,0.0,0.0,0.0,0.0,0.0,13.3,33.9,10.6
4,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,1.5,0.0,0.0,0.0,0.0,0.0,0.0,13.3,33.9,10.3


In [498]:
demographics = demographics.drop_duplicates('SUBJECT_ID', keep='last')

In [499]:
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,VALUENUM-6-5-4-3-2-1,ITEMID-3-2,VALUENUM-6-5-4-3-2,ITEMID-3,VALUENUM-6-5-4-3,ITEMID,VALUENUM-6-5-4,VALUENUM-6-5,VALUENUM-6,VALUENUM
9,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,1.5,0.0,0.0,0.0,0.0,0.0,0.0,13.3,33.9,11.0
13,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,0.8,0.0,0.0,0.0,0.0,0.0,0.0,13.2,24.5,7.1
155,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,55.241096,WHITE,MARRIED,NaN,CATHOLIC,...,0.5,0.0,0.0,0.0,0.0,0.0,0.0,12.2,13.1,4.2
156,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,73.539726,WHITE,SINGLE,NaN,JEWISH,...,1.6,0.0,0.0,0.0,0.0,0.0,0.0,13.2,54.1,18.4
162,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,44.512329,WHITE,MARRIED,NaN,OTHER,...,0.5,0.0,0.0,0.0,0.0,0.0,0.0,12.9,36.3,12.6


In [500]:
# Add white blood cells feature, ITMEID = 51301
eve10 = eve[eve['ITEMID'] == 51301]

In [501]:
demographics.columns = demographics.columns.str.replace('VALUENUM','VALUENUM-7')

In [502]:
demographics = pd.merge(demographics,eve10[['SUBJECT_ID', 'VALUENUM']], on='SUBJECT_ID', how='left' )

In [503]:
demographics = demographics.drop_duplicates('SUBJECT_ID', keep='last')
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,ITEMID-3-2,VALUENUM-7-6-5-4-3-2,ITEMID-3,VALUENUM-7-6-5-4-3,ITEMID,VALUENUM-7-6-5-4,VALUENUM-7-6-5,VALUENUM-7-6,VALUENUM-7,VALUENUM
9,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0.0,0.0,0.0,0.0,0.0,0.0,13.3,33.9,11.0,26.3
13,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,0.0,0.0,0.0,0.0,0.0,0.0,13.2,24.5,7.1,8.2
153,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,55.241096,WHITE,MARRIED,NaN,CATHOLIC,...,0.0,0.0,0.0,0.0,0.0,0.0,12.2,13.1,4.2,0.4
154,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,73.539726,WHITE,SINGLE,NaN,JEWISH,...,0.0,0.0,0.0,0.0,0.0,0.0,13.2,54.1,18.4,9.3
160,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,44.512329,WHITE,MARRIED,NaN,OTHER,...,0.0,0.0,0.0,0.0,0.0,0.0,12.9,36.3,12.6,7.1


In [504]:
# Add platelets feature, ITMEID = 51265
eve11 = eve[eve['ITEMID'] == 51265]

In [505]:
demographics.columns = demographics.columns.str.replace('VALUENUM','VALUENUM-8')

In [506]:
demographics = pd.merge(demographics,eve11[['SUBJECT_ID', 'VALUENUM']], on='SUBJECT_ID', how='left' )

In [507]:
demographics = demographics.drop_duplicates('SUBJECT_ID', keep='last')
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,VALUENUM-8-7-6-5-4-3-2,ITEMID-3,VALUENUM-8-7-6-5-4-3,ITEMID,VALUENUM-8-7-6-5-4,VALUENUM-8-7-6-5,VALUENUM-8-7-6,VALUENUM-8-7,VALUENUM-8,VALUENUM
10,31,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,72.312329,WHITE,MARRIED,NaN,CATHOLIC,...,0.0,0.0,0.0,0.0,0.0,13.3,33.9,11.0,26.3,260.0
14,56,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,...,0.0,0.0,0.0,0.0,0.0,13.2,24.5,7.1,8.2,124.0
207,61,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,55.241096,WHITE,MARRIED,NaN,CATHOLIC,...,0.0,0.0,0.0,0.0,0.0,12.2,13.1,4.2,0.4,11.0
208,67,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,73.539726,WHITE,SINGLE,NaN,JEWISH,...,0.0,0.0,0.0,0.0,0.0,13.2,54.1,18.4,9.3,183.0
214,84,F,2151-10-21,2196-04-17,2196-04-14 04:02:00,44.512329,WHITE,MARRIED,NaN,OTHER,...,0.0,0.0,0.0,0.0,0.0,12.9,36.3,12.6,7.1,262.0


In [508]:
# demographics['VALUENUM-7'] = demographics['VALUENUM-7'].fillna(0)
# demographics['VALUENUM-7-6'] = demographics['VALUENUM-6-5'].fillna(0)
# demographics['VALUENUM-6-5-4'] = demographics['VALUENUM-6-5-4'].fillna(0)
# demographics['VALUENUM-6-5-4-3'] = demographics['VALUENUM-6-5-4-3'].fillna(0)
# demographics['VALUENUM-6-5-4-3-2'] = demographics['VALUENUM-6-5-4-3-2'].fillna(0)
# demographics['VALUENUM-6-5-4-3-2-1'] = demographics['VALUENUM-6-5-4-3-2-1'].fillna(0)
# demographics['VALUENUM'] = demographics['VALUENUM'].fillna(0)

In [509]:
# Split List into those whose age we know, those still alive, and those too old to have age listed
# Combine those still alive with those who died at a known age
Patients_old = demographics[demographics['OLD_FLAG'] == 1]
Patients_young = demographics[demographics['OLD_FLAG'] == 0]
Patients_alive = demographics[demographics['DEATH_FLAG'] == 0]
Patients_dead = demographics[demographics['DEATH_FLAG'] == 1]
print('# of total patients is %d' % len(demographics,))
print('# of old patients is %d' % len(Patients_old))
print('# of young patients is %d' % len(Patients_young))
print('# of living patients is %d' % len(Patients_alive))
print('# of dead patients is %d' % len(Patients_dead))

# of total patients is 46520
# of old patients is 1991
# of young patients is 44529
# of living patients is 33840
# of dead patients is 12680


In [510]:
# Fill na values with "uknown" outside of age
demographics = demographics.fillna({'ETHNICITY':'UKNOWN','MARITAL_STATUS':'UKNOWN', 'RELIGION': 'UKNOWN', 'LANGUAGE':'UKNOWN', 'INSURANCE':'UKNOWN', 'ADMISSION_LOCATION':'UKNOWN'})

In [511]:
demographics.groupby('GENDER').mean()

,SUBJECT_ID,ADMIT_AGE,#ADMISSIONS,delta,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,CANCER_FLAG,AKI_DIAGNOSIS_FLAG,KIDNEY_FAILURE_FLAG,...,VALUENUM-8-7-6-5-4-3-2,ITEMID-3,VALUENUM-8-7-6-5-4-3,ITEMID,VALUENUM-8-7-6-5-4,VALUENUM-8-7-6-5,VALUENUM-8-7-6,VALUENUM-8-7,VALUENUM-8,VALUENUM
GENDER,,,,,,,,,,,,,,,,,,,,,
F,34404.311780,51.874678,1.275847,9641.465651,0.222805,0.286975,0.063434,0.997402,0.045787,0.020099,...,8.849110,232.657287,0.016354,307.828619,0.007005,14.955523,34.598939,11.593531,11.500884,268.346841
M,34442.532751,51.324122,1.261437,10066.030543,0.206807,0.261322,0.026684,0.998354,0.050266,0.019601,...,9.415489,216.858160,0.021814,283.394204,0.009701,15.117011,35.376649,11.991344,11.071753,246.477532


In [512]:
demographics.groupby('INSURANCE').mean()

,SUBJECT_ID,ADMIT_AGE,#ADMISSIONS,delta,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,CANCER_FLAG,AKI_DIAGNOSIS_FLAG,KIDNEY_FAILURE_FLAG,...,VALUENUM-8-7-6-5-4-3-2,ITEMID-3,VALUENUM-8-7-6-5-4-3,ITEMID,VALUENUM-8-7-6-5-4,VALUENUM-8-7-6-5,VALUENUM-8-7-6,VALUENUM-8-7,VALUENUM-8,VALUENUM
INSURANCE,,,,,,,,,,,,,,,,,,,,,
Government,34431.435829,33.601264,1.157086,9051.422027,0.074198,0.105615,0.000668,0.998663,0.032086,0.008690,...,7.762701,102.336898,0.037299,170.628342,0.004278,14.578391,37.050069,12.557953,11.078180,270.873356
Medicaid,34405.221409,35.041305,1.297347,8283.222903,0.126944,0.178866,0.003888,0.999085,0.046661,0.013266,...,11.614822,140.069533,0.009401,210.186642,0.007319,14.956227,36.767861,12.395880,11.031156,270.507809
Medicare,38173.082174,73.243174,1.383479,10144.273615,0.354606,0.431761,0.091352,0.997562,0.066208,0.033176,...,7.938859,287.861561,0.028787,383.152350,0.009895,15.355422,32.196115,10.755059,10.781306,241.827846
Private,30362.033126,35.538944,1.149877,9642.147673,0.095832,0.135844,0.003026,0.998018,0.030831,0.007721,...,9.982837,186.355052,0.010637,231.703480,0.007841,14.634729,37.684349,12.824885,11.873631,267.878447
Self Pay,33697.480496,38.750768,1.046099,3629.492754,0.046099,0.184397,0.007092,0.998227,0.033688,0.015957,...,11.751773,90.482270,0.000709,90.517730,0.001064,14.338958,36.183604,12.366245,11.277081,253.592793


In [513]:
demographics.groupby('#ADMISSIONS').mean()

,SUBJECT_ID,ADMIT_AGE,delta,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,CANCER_FLAG,AKI_DIAGNOSIS_FLAG,KIDNEY_FAILURE_FLAG,HYPO_FLAG,...,VALUENUM-8-7-6-5-4-3-2,ITEMID-3,VALUENUM-8-7-6-5-4-3,ITEMID,VALUENUM-8-7-6-5-4,VALUENUM-8-7-6-5,VALUENUM-8-7-6,VALUENUM-8-7,VALUENUM-8,VALUENUM
#ADMISSIONS,,,,,,,,,,,,,,,,,,,,,
1,34638.924685,49.433908,10947.551282,0.192443,0.237488,0.040941,0.997922,0.036144,0.016366,0.002206,...,6.439679,140.071929,0.011523,178.105123,0.005213,14.897295,35.611333,12.035967,11.411001,257.328244
2,34305.526938,61.926318,7380.385114,0.306783,0.415504,0.051163,0.998062,0.086628,0.030814,0.006589,...,18.053295,445.046512,0.059729,702.459690,0.018702,15.447707,32.450136,10.858525,10.615525,251.611978
3,34086.203428,65.214989,6105.937855,0.373323,0.514158,0.061848,0.998510,0.136364,0.048435,0.008942,...,24.850969,988.697466,0.045306,951.043219,0.025857,15.995116,31.673472,10.516617,10.392230,244.785981
4,29187.505906,64.378128,4794.126712,0.338583,0.545276,0.053150,0.994094,0.200787,0.053150,0.007874,...,29.287402,1105.023622,0.062598,1909.425197,0.056102,16.376285,31.338445,10.325787,10.354626,252.448819
5,25733.471545,63.771361,3994.280844,0.382114,0.597561,0.040650,1.000000,0.207317,0.048780,0.004065,...,63.605691,1037.235772,0.033333,1660.227642,0.071951,16.617480,30.784553,10.140650,10.633740,247.666667
6,25840.283186,63.594674,2535.073380,0.336283,0.601770,0.053097,1.000000,0.168142,0.079646,0.017699,...,47.309735,1354.831858,0.089381,1807.150442,0.061947,15.759292,31.121239,10.313274,9.227434,246.061947
7,28175.392157,60.421096,2658.317857,0.294118,0.529412,0.019608,1.000000,0.098039,0.078431,0.019608,...,54.254902,2001.254902,0.052941,0.000000,0.000000,16.578000,30.947059,10.103922,9.252941,232.941176
8,28306.451613,60.123023,1405.428704,0.387097,0.580645,0.000000,1.000000,0.258065,0.064516,0.032258,...,72.354839,0.000000,0.000000,1646.838710,0.041935,16.674194,31.967742,10.496774,7.725806,201.870968
9,20859.153846,62.096628,1682.958333,0.307692,0.692308,0.000000,1.000000,0.346154,0.000000,0.076923,...,99.230769,5888.307692,1.019231,3927.076923,0.092308,15.300000,30.823077,10.034615,8.453846,223.269231


In [514]:
demographics[demographics['AKI_DIAGNOSIS_FLAG'] == 1].head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,...,VALUENUM-8-7-6-5-4-3-2,ITEMID-3,VALUENUM-8-7-6-5-4-3,ITEMID,VALUENUM-8-7-6-5-4,VALUENUM-8-7-6-5,VALUENUM-8-7-6,VALUENUM-8-7,VALUENUM-8,VALUENUM
376,141,M,2060-05-03,2140-11-09,2140-11-06 11:07:00,80.564384,OTHER,SINGLE,UKNOWN,OTHER,...,0.0,0.0,0.0,0.0,0.0,13.0,25.5,9.1,3.4,144.0
1119,491,M,2090-04-25,2170-04-05,2170-04-02 21:41:00,79.989041,WHITE,MARRIED,UKNOWN,JEWISH,...,0.0,0.0,0.0,0.0,0.0,18.2,30.4,10.7,15.3,128.0
1131,495,M,2106-09-23,2183-05-22,2183-05-13 15:08:00,76.687671,WHITE,SINGLE,UKNOWN,EPISCOPALIAN,...,0.0,0.0,0.0,0.0,0.0,20.2,26.7,8.7,5.5,135.0
1263,502,M,2093-04-27,2143-11-04,2143-10-23 21:05:00,50.520548,WHITE,MARRIED,ENGL,PROTESTANT QUAKER,...,0.0,0.0,0.0,0.0,0.0,13.8,20.4,6.5,25.5,230.0
1920,268,F,2132-02-21,2198-02-18,2198-02-11 13:40:00,66.019178,HISPANIC OR LATINO,SEPARATED,UKNOWN,CATHOLIC,...,0.0,0.0,0.0,0.0,0.0,14.3,30.2,9.7,8.8,145.0


In [515]:
demographics.groupby('CANCER_FLAG').mean()

,SUBJECT_ID,ADMIT_AGE,#ADMISSIONS,delta,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,AKI_DIAGNOSIS_FLAG,KIDNEY_FAILURE_FLAG,HYPO_FLAG,...,VALUENUM-8-7-6-5-4-3-2,ITEMID-3,VALUENUM-8-7-6-5-4-3,ITEMID,VALUENUM-8-7-6-5-4,VALUENUM-8-7-6-5,VALUENUM-8-7-6,VALUENUM-8-7,VALUENUM-8,VALUENUM
CANCER_FLAG,,,,,,,,,,,,,,,,,,,,,
0,32508.364583,65.048472,1.239583,230.859722,0.000000,1.000000,0.052083,0.093750,0.010417,0.020833,...,0.000000,1063.166667,0.011458,1595.375000,0.097917,16.237234,31.177895,10.266316,15.829474,254.515789
1,34429.737873,51.532715,1.267814,9930.197651,0.214264,0.271067,0.042780,0.048208,0.019839,0.003080,...,9.186089,222.050319,0.019436,291.417801,0.008334,15.044358,35.044381,11.820538,11.249904,256.047874


In [516]:
demographics[demographics['AKI_DIAGNOSIS_FLAG'] == 1].groupby('#ADMISSIONS').size()

#ADMISSIONS
1     1409
2      447
3      183
4      102
5       51
6       19
7        5
8        8
9        9
10       4
11       3
12       1
13       2
14       3
16       1
dtype: int64

In [517]:
demographics.groupby('AKI_DIAGNOSIS_FLAG').mean()

,SUBJECT_ID,ADMIT_AGE,#ADMISSIONS,delta,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,CANCER_FLAG,KIDNEY_FAILURE_FLAG,HYPO_FLAG,...,VALUENUM-8-7-6-5-4-3-2,ITEMID-3,VALUENUM-8-7-6-5-4-3,ITEMID,VALUENUM-8-7-6-5-4,VALUENUM-8-7-6-5,VALUENUM-8-7-6,VALUENUM-8-7,VALUENUM-8,VALUENUM
AKI_DIAGNOSIS_FLAG,,,,,,,,,,,,,,,,,,,,,
0,34117.874890,50.897010,1.242247,10362.382595,0.211732,0.257584,0.042983,0.998035,0.017076,0.002778,...,7.898042,182.121293,0.015797,249.073521,0.006370,14.921082,35.290646,11.910361,11.212583,257.937886
1,40492.336004,64.577996,1.770360,4828.473386,0.255007,0.567868,0.039163,0.995995,0.073876,0.009791,...,34.172230,1044.713841,0.090788,1181.443703,0.050868,17.047749,30.153761,10.033348,12.159470,219.772586


In [518]:
demographics.groupby('MARITAL_STATUS').mean()

,SUBJECT_ID,ADMIT_AGE,#ADMISSIONS,delta,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,CANCER_FLAG,AKI_DIAGNOSIS_FLAG,KIDNEY_FAILURE_FLAG,...,VALUENUM-8-7-6-5-4-3-2,ITEMID-3,VALUENUM-8-7-6-5-4-3,ITEMID,VALUENUM-8-7-6-5-4,VALUENUM-8-7-6-5,VALUENUM-8-7-6,VALUENUM-8-7,VALUENUM-8,VALUENUM
MARITAL_STATUS,,,,,,,,,,,,,,,,,,,,,
DIVORCED,40009.621193,62.288776,1.375804,10777.570580,0.272844,0.323466,0.017160,0.995710,0.070356,0.015873,...,9.206778,240.820249,0.019305,262.815959,0.008795,15.090171,32.402964,10.885260,10.658689,255.401375
LIFE PARTNER,63181.076923,55.179136,1.461538,20689.975000,0.153846,0.076923,0.000000,1.000000,0.230769,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,14.441667,30.869231,10.369231,13.553846,219.692308
MARRIED,37930.428239,64.897456,1.306078,10292.129030,0.243132,0.305862,0.027471,0.996864,0.057214,0.022658,...,11.137465,331.161583,0.031484,383.745836,0.012611,15.043118,32.613928,11.007474,10.485415,243.023808
SEPARATED,36035.367188,58.376208,1.513021,7806.029348,0.218750,0.312500,0.010417,1.000000,0.070312,0.023438,...,20.940104,132.895833,0.001042,398.843750,0.008333,14.925401,32.999740,11.002872,9.782786,254.127604
SINGLE,39706.465547,49.975194,1.360397,8880.341986,0.193611,0.249104,0.024368,0.998669,0.057029,0.017815,...,15.542644,271.696939,0.022658,365.889219,0.009368,14.961061,33.482755,11.297002,10.278326,260.063711
UKNOWN,19794.865834,13.009477,1.035436,8621.851202,0.055817,0.102827,0.013417,0.999795,0.010754,0.008296,...,0.854056,26.132732,0.000451,26.142974,0.000543,14.726567,44.566339,15.146530,14.394574,286.784472
UNKNOWN (DEFAULT),32384.394137,65.562132,1.117264,10362.023810,0.335505,0.436482,0.068404,1.000000,0.042345,0.022801,...,13.205212,0.000000,0.000000,498.879479,0.009446,14.575421,32.848197,10.957049,11.660984,242.095082
WIDOWED,36822.406628,77.708090,1.332296,10106.879714,0.400952,0.471988,0.192054,0.997620,0.058587,0.035701,...,4.906811,140.146466,0.009172,336.483339,0.007268,15.364592,32.095154,10.690283,10.946017,245.254405


In [519]:
demographics.groupby('ETHNICITY').mean()

,SUBJECT_ID,ADMIT_AGE,#ADMISSIONS,delta,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,CANCER_FLAG,AKI_DIAGNOSIS_FLAG,KIDNEY_FAILURE_FLAG,...,VALUENUM-8-7-6-5-4-3-2,ITEMID-3,VALUENUM-8-7-6-5-4-3,ITEMID,VALUENUM-8-7-6-5-4,VALUENUM-8-7-6-5,VALUENUM-8-7-6,VALUENUM-8-7,VALUENUM-8,VALUENUM
ETHNICITY,,,,,,,,,,,,,,,,,,,,,
AMERICAN INDIAN/ALASKA NATIVE,22393.636364,22.282939,1.136364,13932.022917,0.113636,0.113636,0.000000,1.000000,0.022727,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,17.705882,41.297674,14.027907,11.927907,278.627907
AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE,48575.000000,52.306849,1.500000,4971.600000,0.500000,1.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,30.500000,29.850000,9.350000,11.550000,205.000000
ASIAN,23682.855050,26.569184,1.158057,6215.750903,0.104086,0.165767,0.020046,0.996145,0.018504,0.009252,...,4.569005,118.038551,0.002621,118.084811,0.001696,14.440620,41.447623,14.019086,13.533675,274.070732
ASIAN - ASIAN INDIAN,54308.413793,52.082094,1.568966,1451.423810,0.068966,0.120690,0.034483,1.000000,0.086207,0.000000,...,11.379310,0.000000,0.000000,880.206897,0.022414,14.050943,32.614035,10.856140,10.945614,292.964912
ASIAN - CAMBODIAN,61233.909091,54.512827,1.636364,646.002778,0.181818,0.545455,0.000000,1.000000,0.272727,0.090909,...,65.909091,0.000000,0.000000,0.000000,0.000000,14.972727,33.381818,11.136364,12.681818,265.454545
ASIAN - CHINESE,49858.337662,52.959503,1.233766,4542.416049,0.116883,0.212121,0.030303,1.000000,0.051948,0.021645,...,2.753247,220.917749,0.019913,1105.021645,0.016450,15.544022,35.463877,11.896018,10.664558,252.486607
ASIAN - FILIPINO,56435.933333,53.490411,1.666667,3413.837500,0.133333,0.266667,0.066667,1.000000,0.200000,0.066667,...,0.000000,0.000000,0.000000,0.000000,0.000000,17.450000,34.020000,11.286667,9.853333,225.400000
ASIAN - JAPANESE,58800.428571,56.310502,1.000000,90.408333,0.000000,0.285714,0.142857,1.000000,0.000000,0.000000,...,120.000000,0.000000,0.000000,0.000000,0.000000,14.116667,37.371429,12.428571,10.425714,181.857143
ASIAN - KOREAN,56058.166667,53.418721,1.166667,408.908333,0.083333,0.166667,0.000000,1.000000,0.083333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,15.118182,31.833333,11.008333,13.034167,192.166667


In [520]:
demographics.groupby('ADMISSION_LOCATION').mean()

,SUBJECT_ID,ADMIT_AGE,#ADMISSIONS,delta,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,CANCER_FLAG,AKI_DIAGNOSIS_FLAG,KIDNEY_FAILURE_FLAG,...,VALUENUM-8-7-6-5-4-3-2,ITEMID-3,VALUENUM-8-7-6-5-4-3,ITEMID,VALUENUM-8-7-6-5-4,VALUENUM-8-7-6-5,VALUENUM-8-7-6,VALUENUM-8-7,VALUENUM-8,VALUENUM
ADMISSION_LOCATION,,,,,,,,,,,,,,,,,,,,,
** INFO NOT AVAILABLE **,21137.763819,0.834994,1.000000,5471.673333,0.010050,0.020101,0.005025,1.000000,0.000000,0.005025,...,0.000000,0.000000,0.000000,0.000000,0.000000,14.380000,48.836364,16.510286,16.605284,319.925714
CLINIC REFERRAL/PREMATURE,50512.374061,50.014977,1.385879,4799.879529,0.145018,0.241462,0.049775,0.998097,0.058788,0.016525,...,9.518778,112.439059,0.009004,316.997897,0.009354,15.032343,34.905270,11.777941,10.396142,254.644950
EMERGENCY ROOM ADMIT,30047.632710,61.286446,1.329435,9128.245901,0.308199,0.404284,0.070848,0.998276,0.049243,0.029792,...,12.157146,223.025483,0.017617,329.955681,0.008451,14.986150,33.277388,11.193517,10.729685,254.433128
HMO REFERRAL/SICK,15715.180000,0.611178,1.000000,407.745833,0.010000,0.040000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,15.312500,48.965823,16.651899,16.940000,291.531646
PHYS REFERRAL/NORMAL DELI,27924.019579,35.299973,1.149346,16739.021818,0.113086,0.095231,0.006657,0.998747,0.023259,0.002506,...,6.284752,299.741562,0.023377,135.936095,0.003493,14.932303,39.034849,13.209387,12.618820,263.939213
TRANSFER FROM HOSP/EXTRAM,34278.477194,64.160154,1.166236,12182.643196,0.275961,0.331182,0.029834,0.995410,0.075875,0.032846,...,7.232071,248.865175,0.027223,453.990820,0.016394,15.282173,32.069129,10.825852,11.185920,246.077622
TRANSFER FROM OTHER HEALT,36169.980000,64.818712,2.240000,4394.273232,0.280000,0.620000,0.060000,0.980000,0.220000,0.020000,...,6.080000,0.000000,0.000000,0.000000,0.000000,16.316000,30.964000,10.088000,11.148000,234.460000
TRANSFER FROM SKILLED NUR,28537.748718,74.313027,1.610256,6959.658122,0.548718,0.702564,0.235897,1.000000,0.117949,0.051282,...,15.020513,523.405128,0.200000,1309.025641,0.034359,16.355497,31.949485,10.486082,11.469072,268.515464
TRSF WITHIN THIS FACILITY,66104.000000,49.872603,2.000000,NaN,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,11.900000,34.625000,12.025000,6.375000,244.000000


In [521]:
lang = demographics.groupby('LANGUAGE').mean()
language = pd.DataFrame()
language['English'] = lang.loc['ENGL']
lang = lang.drop('ENGL')
language['Others'] = lang.mean()
language

,English,Others
SUBJECT_ID,53104.530700,44497.367518
ADMIT_AGE,61.644550,68.754632
#ADMISSIONS,1.383743,1.408008
delta,6576.354536,6944.415372
OUTSIDE_DEATH_FLAG,0.192193,0.230472
DEATH_FLAG,0.264248,0.376109
OLD_FLAG,0.049782,0.115613
CANCER_FLAG,0.998530,0.994547
AKI_DIAGNOSIS_FLAG,0.059885,0.070915
KIDNEY_FAILURE_FLAG,0.016533,0.069720


In [522]:
# Export data set
demographics.to_csv('Demographics-new.csv')